In [1]:
import pickle # I'm using pickle to save the StandardScaler and the MinMaxScaler

import numpy as np
from keras.models import load_model

from dataset_downloader import get_month_data # this is a function I created to download a mothly data from idojarasbudapest.hu

Using TensorFlow backend.


I created this function to make predictions for 1/7/30 days from a given date:

In [2]:
def make_prediction_for_n_day(n, month, day_of_month):
    mmscaler_file = 'data/mmscaler_{}_days.sav'.format(n) 
    sscaler_file = 'data/sscaler_{}_days.sav'.format(n) 
    sscaler = pickle.load(open(sscaler_file, 'rb')) # loading the standardmaxscaler we used when training the n-days predictor
    mmscaler = pickle.load(open(mmscaler_file, 'rb')) # loading the minmaxscaler we used when training the n-days predictor
    
    data = np.array(get_month_data(2018, month)) # download the data of the given month
    data_of_day = data[day_of_month-1, 1:4] # getting the data of the desired day
    data_of_day = sscaler.transform(data_of_day.reshape(1, -1)) # we trsansform it using standardcaler
    
    model = load_model('data/model_{}_days'.format(n)) # we load the model trained before
    prediction = model.predict(data_of_day) # making prediction
    mean_temperature = mmscaler.inverse_transform(prediction) # inverse transforming the prediction
    
    return float(mean_temperature)
    

### Making predictions for 26. october, 1. november and 29. november. 
For this we download the data of 25. october and predict for 1, 7 and 30 days.

In [3]:
t_mean_26_oct = make_prediction_for_n_day(1, '10', 25)
t_mean_1_nov = make_prediction_for_n_day(7, '10', 25)
t_mean_29_nov = make_prediction_for_n_day(30, '10', 25)

print('The mean temperature on 26. october  will be {0:.2f}°C'.format(t_mean_26_oct))
print('The mean temperature on 1.  november will be {0:.2f}°C'.format(t_mean_1_nov))
print('The mean temperature on 29. november will be {0:.2f}°C'.format(t_mean_29_nov))

A:\PortableApps\WinPython\python-3.6.5.amd64\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


The mean temperature on 26. october  will be 8.48°C
The mean temperature on 1.  november will be 7.21°C
The mean temperature on 29. november will be 11.49°C
